## Part 3: Optimize Matrix Multiplication

#### Goal: Minimize the energy and the area of caches

In this part, we will design both the loop nest for matrix multiplication and the caches, to minimize the energy and the area of caches while correctly computing matrix multiplication. 

#### Matrix Multiplication Problem

$$
C_{m,n} = \sum_k A_{m,k} \times B_{k,n}
$$

where A is a $M\times K$ matrix and B is a $K\times N$ matrix. The matrix dimensions for our problem are $M=64, K=256, N=64$. You can write your own loop nest with any loop orders and tiling strategies that produce a correct result. 

#### Constraints on Cache Designs

You can use any number of different caches (each with a __minimum size of 64 words__). You can have a hierarchy of caches, or multiple small caches, or a single large cache as you like. However the hard constraint on cache design is on the total area and energy of all caches you use:

- Total Area: $20000 \mu m^2$  
- Total Energy: $1 m\text{J}$

You are not allowed to use a register for storing partial sums for this question. Thus, for each accumulate operation (```z[m][n] += a[m][k] * b[k][n]```), you must have one read and one write. The purpose of this question is to explore changing the addressing, cache sizes and tiling strategy. There are multiple possible solutions to achieve a full score. 

Please do not modify the setup code (such as ```z = z_MN.getRoot()```) nor perform your own matrix multiplication using numpy arrays. You are allowed to do this for debugging purposes, but for the final answer please utilize the setup we provide.

#### Important Notes about Energy and Area
Note the following facts about area.
- Larger caches use more area.
- Set-associative caches use more area than direct-mapped caches for the same total capacity due to overheads.

Note the following facts about energy.
- Here, we measure the total energy: the sum of cache and off-chip memory energies.
- Off-chip memory reads and writes cost more energy than cache reads and writes.
- A single read/write of a larger cache costs more energy than a read/write of a smaller cache.
- A single read/write of a set-associative cache costs more energy than direct-mapped cache.

#### Grading

First, clearly explain your assumptions and loop nest details in the cell below. Also, make sure you explain assumptions for your memory layout (e.g., row-major or column-major) and cache designs. You can write a pseudo-code to explain your loop ordering and tiling strategies. Please also explain at which part of the loop nest you are expecting memory access (load and store). 

Second, make sure you correctly write `getAddress` function that will be used to compute memory address. This function should reflect your cache design.

Third, report your total cache area and energy. Grading will be based on these metrics with maximum score of 10 for the total area, and 10 for the total energy:

| Score | Area ($\mu m^2$) | Energy ($m\text{J}$) |
| ---   | ----             | ----                 |
| + 10  | < 12000          |  < 0.65              |
| + 8   | 12000 ~ 14000    |  0.65 ~ 0.75         |
| + 6   | 14000 ~ 16000    |  0.75 ~ 0.85         |
| + 4   | 16000 ~ 18000    |  0.85 ~ 0.95         |
| + 2   | 18000 ~ 20000    |  0.95 ~ 1.0          |

Lastly, breakdown the total energy for each cache you used, and analyze which operation consumes the most energy. Is there any method to improve that operation? You do not need to implement any codes, but please conceptually explain what modifications can be made to caches to reduce energy further. 

In [ ]:
# Run boilerplate code to set up environment
from copy import deepcopy
import numpy as np
import random
import tqdm
from loaders import *

from cache import Cache, CacheAssoc

%run ./prelude.py --style=uncompressed --animation=none

In [ ]:
density = [1.0]
seed = 10

enable_log = False

def set_params(**kwargs):
    global enable_log
    
    for variable, value in kwargs.items():
        globals()[variable] = value

    enable_log = (kwargs["log"] == 'enable')


def logger(arg):
    if enable_log:
        print(arg)

#### Create Input Tensors

Given shapes selected above the below codeblock creates and displays the filter weights (**f**) and input activations (**i**) and a reference output (**o_verify**)

In [ ]:
M = 64
K = 256
N = 64

a_MK_raw = []
for m in range(M):
    a_MK_raw.append([random.randint(1, 9) for i in range(K)])
                 
b_KN_raw = []
for k in range(K):
    b_KN_raw.append([random.randint(1, 9) for i in range(N)])

a_MK = Tensor.fromUncompressed(["M", "K"], a_MK_raw)
b_KN = Tensor.fromUncompressed(["K", "N"], b_KN_raw)

a_MK.setName("A_MK").setColor("blue")
b_KN.setName("B_KN").setColor("green")

# print("Input A")
# displayTensor(a_MK)
                    
# print("Input B")
# displayTensor(b_KN)

z_verify = None

def create_z():
    """
    Create a fully populated z tensor
    """
    z = Tensor(rank_ids=["M", "N"], default='')
    z.setName("Z")
    z.setMutable(True)

    z_m = z.getRoot()
    #
    # Hack to fill in all the entries in z
    # This allows us to pretend the tensor is dense
    #
    n_fiber = Fiber(coords=range(N), initial=1)
    m_fiber = Fiber(coords=range(M), initial=1)

    for m, (z_n, _) in z_m << m_fiber:
        for n, (z_ref, _) in z_n << n_fiber:
            z_ref <<= 0
            
    return z

#### Get ground truth for the result

In [ ]:
print("Matrix Multiply")

z_MN = create_z()

# print("Output - before")
# displayTensor(z_MN)

z = z_MN.getRoot()
a = a_MK.getRoot()
b = b_KN.getRoot()

# Progress bar since this takes a while
pbar = tqdm.tqdm(desc='Progress', total=M)

# canvas = createCanvas(a_MK, b_KN, z_MN)
for m in range(M):
    pbar.update(1)
    a_tile = [ (m, kt) for kt in range(K)]
    for n in range(N):
        logger(f"Processing Z({m},{n}) = {z[m][n]}")
        b_tile = [ (kt, n) for kt in range(K)]
        z_tile = (m, n)
        for k in range(K):
            logger(f"Processing A({m},{k}) = {a[m][k].payload}")
            logger(f"Processing B({k},{n}) = {b[k][n].payload}")
            
            z[m][n] += a[m][k] * b[k][n]
            # addActivity(canvas, a_tile, b_tile, z_tile, worker="W")
            # addFrame(canvas, (m,k), (k,n), (m,n))
print('Done!')  
pbar.close()

# print("Output - after")
# displayTensor(z)

# displayCanvas(canvas)

if z_verify is None:
    z_verify = deepcopy(z)

## Question 9 

Modify caches and loop nest below for your design. Run the cache profiler, and report the total area and energy. 

#### Your Code: Utility function for addressing

In [ ]:
# Modify this function if necessary (e.g., different memory addressing)

def getAddress(tensor, x, y):
    return (x*tensor.getShape()[1]+y)

#### Your Code: Caches

You can use direct-mapped caches or set-associative caches.

In [ ]:
# Define your caches; minimum size is log_size=6
# You can use any number of separate caches, as far as they meet area and energy constraints

cache_a = Cache(log_size=6, words_per_line=1)
cache_b = Cache(log_size=6, words_per_line=1)
cache_c = Cache(log_size=6, words_per_line=1)

# List all caches you are using
caches = [cache_a, cache_b, cache_c]

#### Your Code: Tiling and Loop Nest

In [ ]:
# Tiling Size: Modify this to change tiling

# M = M1 x M0
# K = K1 x K0
# N = N1 x N0

M1 = 8
M0 = 8
K1 = 16
K0 = 16
N1 = 8
N0 = 8

print("Matrix Multiply")

z_MN = create_z()

# print("Output - before")
# displayTensor(z_MN)

z = z_MN.getRoot()
a = a_MK.getRoot()
b = b_KN.getRoot()

# canvas = createCanvas(a_MK, b_KN, z_MN)

# Progress bar since this takes a while
pbar = tqdm.tqdm(desc='Progress', total=K1*M1*N1)

# Loop Nest: Modify this to alter the for loop orders
for k1 in range(K1):
    for m1 in range(M1):
        for n1 in range(N1):
            pbar.update(1)
            
            a_tile = [ (m1*M0+mt, k1*K0+kt) for mt in range(M0)for kt in range(K0)]
            b_tile = [ (k1*K0+kt, n1*N0+nt) for kt in range(K0)for nt in range(N0)]
            z_tile = [ (m1*M0+mt, n1*N0+nt) for mt in range(M0)for nt in range(N0)]

            for m0 in range(M0):
                for n0 in range(N0):
                    for k0 in range(K0):

                        m = m1*M0+m0
                        n = n1*N0+n0
                        k = k1*K0+k0

                        logger(f"Processing A({m},{k}) = {a[m][k].payload}")
                        logger(f"Processing B({k},{n}) = {b[k][n].payload}")
                        logger(f"Processing Z({m},{n}) = {z[m][n]}")

                        cache_c.load(getAddress(z, m, n))
                        cache_a.load(getAddress(a, m, k))
                        cache_b.load(getAddress(b, k, n))

                        z[m][n] += a[m][k] * b[k][n]

                        # addActivity(canvas, a_tile, b_tile, z_tile, worker="W")
                        # addFrame(canvas, (m,k), (k,n), (m,n))

                        cache_c.store(getAddress(z, m, n))
print('Done!')
pbar.close()
# print("Output - after")
# displayTensor(z)

# displayCanvas(canvas)

if z_verify is None:
    print("Result not verified")
else:
    assert z == z_verify

# Print cache statistics
print("-------Cache A--------")
cache_a.print_stats()
print("-------Cache B--------")
cache_b.print_stats()
print("-------Cache C--------")
cache_c.print_stats()

#### Area / Energy Profiler

In [ ]:
# You do not need to change any of this part.
from cache_profiler import CacheProfiler

res_list = []
for cache in caches:
    profiler = CacheProfiler(cache)
    res = profiler.profile(cache.stats)
    res_list.append(res)

total_energy = 0
total_area = 0
for i, res in enumerate(res_list):
    print(f"Cache {i+1} Energy: {res['energy'] / 1e9 :.4f} mj, Area: {res['area'] :.2f} um^2")
    total_energy += res['energy']
    total_area += res['area']

total_energy /= 1e9 # Convert to mJ
print(f"Total Energy: {total_energy :.4f} mj, Area: {total_area :.2f} um^2")

answer( # DO NOT MODIFY
    question='3.1.1',
    subquestion='',
    answer=total_energy<1.0,
    required_type=bool,
)
answer( # DO NOT MODIFY
    question='3.1.1',
    subquestion='',
    answer=total_energy<0.95,
    required_type=bool,
)
answer( # DO NOT MODIFY
    question='3.1.1',
    subquestion='',
    answer=total_energy<0.85,
    required_type=bool,
)
answer( # DO NOT MODIFY
    question='3.1.1',
    subquestion='',
    answer=total_energy<0.75,
    required_type=bool,
)
answer( # DO NOT MODIFY
    question='3.1.1',
    subquestion='',
    answer=total_energy<0.65,
    required_type=bool,
)
answer( # DO NOT MODIFY
    question='3.1.2',
    subquestion='',
    answer=total_area<20000,
    required_type=bool,
)
answer( # DO NOT MODIFY
    question='3.1.2',
    subquestion='',
    answer=total_area<18000,
    required_type=bool,
)
answer( # DO NOT MODIFY
    question='3.1.2',
    subquestion='',
    answer=total_area<16000,
    required_type=bool,
)
answer( # DO NOT MODIFY
    question='3.1.2',
    subquestion='',
    answer=total_area<14000,
    required_type=bool,
)
answer( # DO NOT MODIFY
    question='3.1.2',
    subquestion='',
    answer=total_area<12000,
    required_type=bool,
)